In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from arch import arch_model

import warnings
warnings.filterwarnings("ignore", message=".*no associated frequency information.*")

# === Load and preprocess data ===
df = pd.read_csv("/Users/arifpras/Library/CloudStorage/OneDrive-Kemenkeu/01_Kemenkeu/TK4_202512/20251215_priceyield.csv")

# # Filter from Jan 2015
# df = df[df['date'] >= '2015-01-01']

In [2]:
# Inspect DataFrame structure
print("Shape:", df.shape)
print("\nColumns:\n", df.columns.tolist())
print("\nDtypes:")
print(df.dtypes)

print("\nInfo:")
# df.info prints to stdout; capture it
df_info = None
try:
    df.info()
except Exception:
    pass

print("\nHead:")
display(df.head())

print("\nDescribe (numeric):")
display(df.describe(include='number').T)

print("\nMissing values per column:")
print(df.isnull().sum())

print("\nMemory usage (bytes per column):")
print(df.memory_usage(deep=True))

# Extra checks for date-like columns
for col in df.columns:
    if 'date' in col.lower():
        print(f"\nColumn '{col}' looks like a date column. Parsing to datetime...")
        parsed = pd.to_datetime(df[col], errors='coerce')
        print("Nulls after parse:", parsed.isnull().sum())
        print("Min, Max:", parsed.min(), parsed.max())

print("\nSample rows:")
display(df.sample(5, random_state=1))

Shape: (1540, 8)

Columns:
 ['date', 'cusip', 'series', 'coupon', 'maturity_date', 'price', 'yield', 'tenor']

Dtypes:
date              object
cusip             object
series            object
coupon           float64
maturity_date     object
price            float64
yield            float64
tenor             object
dtype: object

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1540 non-null   object 
 1   cusip          1540 non-null   object 
 2   series         1540 non-null   object 
 3   coupon         1540 non-null   float64
 4   maturity_date  1540 non-null   object 
 5   price          1385 non-null   float64
 6   yield          1385 non-null   float64
 7   tenor          1540 non-null   object 
dtypes: float64(3), object(5)
memory usage: 96.4+ KB

Head:


,date,cusip,series,coupon,maturity_date,price,yield,tenor
0,02/01/2023,BY4236745,FR95,6.375,15/08/2028,99.102,6.567,05_year
1,03/01/2023,BY4236745,FR95,6.375,15/08/2028,98.710,6.652,05_year
2,04/01/2023,BY4236745,FR95,6.375,15/08/2028,98.845,6.623,05_year
3,05/01/2023,BY4236745,FR95,6.375,15/08/2028,98.989,6.592,05_year
4,06/01/2023,BY4236745,FR95,6.375,15/08/2028,98.945,6.602,05_year



Describe (numeric):


,count,mean,std,min,25%,50%,75%,max
coupon,1540.0,6.688718,0.214572,6.375,6.500,6.6875,6.875,7.000
price,1385.0,100.851023,2.111788,95.873,99.449,100.5090,102.115,106.029
yield,1385.0,6.528587,0.383751,5.305,6.336,6.5830,6.818,7.256



Missing values per column:
date               0
cusip              0
series             0
coupon             0
maturity_date      0
price            155
yield            155
tenor              0
dtype: int64

Memory usage (bytes per column):
Index              132
date             90860
cusip            89320
series           82638
coupon           12320
maturity_date    90860
price            12320
yield            12320
tenor            86240
dtype: int64

Column 'date' looks like a date column. Parsing to datetime...
Nulls after parse: 922
Min, Max: 2023-01-02 00:00:00 2025-12-12 00:00:00

Column 'maturity_date' looks like a date column. Parsing to datetime...
Nulls after parse: 0
Min, Max: 2028-08-15 00:00:00 2035-07-15 00:00:00

Sample rows:


/var/folders/9k/ff8g4yqn45s139zzyc224r800000gn/T/ipykernel_13572/3217426080.py:31: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  parsed = pd.to_datetime(df[col], errors='coerce')


,date,cusip,series,coupon,maturity_date,price,yield,tenor
894,23/06/2023,BY4236786,FR96,7.000,15/02/2033,105.147,6.278,10_year
1205,02/09/2024,ZI4194448,FR100,6.625,15/02/2034,99.900,6.639,10_year
1045,22/01/2024,ZI4194448,FR100,6.625,15/02/2034,100.279,6.586,10_year
422,14/08/2024,ZH8062890,FR101,6.875,15/04/2029,101.149,6.582,05_year
909,14/07/2023,BY4236786,FR96,7.000,15/02/2033,106.008,6.159,10_year


In [4]:
# Convert `date` to datetime (originally DD/MM/YYYY) and reformat to MM/DD/YYYY
print("Before conversion dtypes:\n", df.dtypes[['date','maturity_date']])

# Parse using dayfirst=True to handle DD/MM/YYYY
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
# Also ensure maturity_date is datetime
df['maturity_date'] = pd.to_datetime(df['maturity_date'], dayfirst=True, errors='coerce')

print("\nNulls after parsing `date`:", df['date'].isnull().sum())

# Reformat `date` to string format MM/DD/YYYY (if you prefer to keep datetime objects, skip this step)
df['date'] = df['date'].dt.strftime('%m/%d/%Y')

print("\nAfter conversion dtypes:\n", df.dtypes[['date','maturity_date']])
print("\nSample `date` values:")
print(df['date'].head().to_list())

# Save cleaned DataFrame to CSV
out_path = "/Users/arifpras/Library/CloudStorage/OneDrive-Kemenkeu/01_Kemenkeu/TK4_202512/20251215_priceyield_cleaned.csv"
df.to_csv(out_path, index=False)
print(f"Saved DataFrame with shape {df.shape} to {out_path}")

# Show a quick head to confirm
display(df.head())

Before conversion dtypes:
 date                     object
maturity_date    datetime64[ns]
dtype: object

Nulls after parsing `date`: 922

After conversion dtypes:
 date                     object
maturity_date    datetime64[ns]
dtype: object

Sample `date` values:
['02/01/2023', '03/01/2023', '04/01/2023', '05/01/2023', '06/01/2023']
Saved DataFrame with shape (1540, 8) to /Users/arifpras/Library/CloudStorage/OneDrive-Kemenkeu/01_Kemenkeu/TK4_202512/20251215_priceyield_cleaned.csv


,date,cusip,series,coupon,maturity_date,price,yield,tenor
0,02/01/2023,BY4236745,FR95,6.375,2028-08-15,99.102,6.567,05_year
1,03/01/2023,BY4236745,FR95,6.375,2028-08-15,98.710,6.652,05_year
2,04/01/2023,BY4236745,FR95,6.375,2028-08-15,98.845,6.623,05_year
3,05/01/2023,BY4236745,FR95,6.375,2028-08-15,98.989,6.592,05_year
4,06/01/2023,BY4236745,FR95,6.375,2028-08-15,98.945,6.602,05_year
